# Homework 6 – Models and Pipelines 🔁

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from hw import *

In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.express as px

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


from plotly.subplots import make_subplots
import plotly.graph_objects as go

import warnings
warnings.simplefilter('ignore')

## Part 1: `sklearn` Pipelines 🧠

The file `data/toy.csv` contains an example dataset that consists of 4 columns:

- `'group'`: a categorical column with 3 categories
- `'c1'`: a numeric attribute
- `'c2'`: a numeric attribute
- `'y'`: the target variable (that you want to predict) 
```

In the following questions, you will build `Pipeline`s that combine feature engineering with a linear regression model.

In [ ]:
fp = os.path.join('data', 'toy.csv')
data = pd.read_csv(fp)
data.head()

### Question 1

First, you will train a regression model using only a *log-scaled* `'c2'` variable. Create a `Pipeline` that:
1. log-scales `'c2'`, then
2. predicts `'y'` using a linear regression model (using your transformed `'c2'`).

That is, complete the implementation of the function `simple_pipeline`, which takes in a DataFrame like `data` and returns a **tuple** consisting of 
- An already-fit `Pipeline`, and
- An array containing the predictions your model makes on `data` (after being trained on `data`).

***Note:*** By "log", we're referring to the natural logarithm. In order to log-scale `'c2'`, you'll need to use a `FunctionTransformer`.

In [ ]:
# don't change this cell, but do run it 
q1_fp = os.path.join('data', 'toy.csv')
q1_data = pd.read_csv(q1_fp)
q1_pl, q1_preds = simple_pipeline(q1_data)

### Question 2

Now, you will engineer features from the other columns and use them to train a regression model.  Create a `Pipeline` that:
1. uses `'c1'` as is,
1. log-scales `'c2'`,
1. one-hot encodes `'group'`, and
1. predicts `'y'` using a linear regression model built on the three variables above. (Note that your model will have more than three "features", because one-hot encoding `'group'` will create multiple columns. Don't drop any of them.)

That is, complete the implementation of the function `multi_type_pipeline`, which takes in a DataFrame like `data` and returns a **tuple** consisting of
- An already-fit `Pipeline`, and
- An array containing the predictions your model makes on `data` (after being trained on `data`).

***Hint:*** Use `ColumnTransformer`

In [ ]:
# don't change this cell, but do run it 
q2_fp = os.path.join('data', 'toy.csv')
q2_data = pd.read_csv(q2_fp)
q2_pl, q2_preds = multi_type_pipeline(q2_data)

### Question 3

It seems like `'c1'` and `'c2'` have strong associations with the values of `'group'` (to see this, run the cell below). This suggests that group-wise scaling might make good features. 


Now, we want to standardize (i.e. z-score) both `'c1'` and `'c2'` **within each `'group'`** (`'A'`, `'B'`, and `'C'`). Unfortunately, there is no built-in transformer in `sklearn` that performs group-wise standardization, so **you will need to create your own transformer!**

Your job is to complete the implementation of the `StdScalerByGroup` transformer class, meaning that you need to implement the `fit` and `transform` methods, along with the constructor (`__init__`).
- The `StdScalerByGroup` transformer works on an input array/DataFrame `X` whose first column contains groups, and whose remaining columns are quantitative and need to be standardized (within each group).
- The `fit` method should determine the mean and standard deviation of each quantitative column within each group in the input data `X` and save them in the instance variable `grps_`. (For instance, one of the quantities you may calculate here is the standard deviation of `'c1'`, but only for the rows whose `'group'` is `'B'`.)
- The `transform` method should take in an input array/DataFrame `X`, standardize each quantitative column separately using the means and standard deviations stored in `grps_`, and return a DataFrame containing the transformed quantitative columns.


If you `fit` and `transform` a `StdScalerByGroup` transformer on the `toy` DataFrame (without the `'y'` column), you should get back a DataFrame with two columns, `'c1'` and `'c2'`, with groups stored in the index (if you end up creating a `MultiIndex`, that is fine).


***Notes:***
1. You may decide on whatever structure you'd like for the `grps_` variable. This question will be graded on the correctness of the output of your transformer. (Check the correctness of your work by checking the output by-hand!)    
2. At no point should you loop over the **rows** of `data` (in fact, our solution doesn't use any loops).
3. The `'group'` column in the public tests is named `'g'` instead of `'group'`. Remember, the first column will **always** contain the groups, even if the first column's name is something other than `'group'`.
4. Do not worry about cases where the standard deviation is equal to 0, the function `std` might work in this case.

In [ ]:
# The scatter plot referenced at the start of Question 3
# This is not needed to answer the question, but motivates why we are standardizing
px.scatter(data, x='c1', y='y', color='group')

In [ ]:
# don't change this cell, but do run it 
# test fit 
q3_test_fit_cols = {'g': ['A', 'A', 'B', 'B'], 'c1': [1, 2, 2, 2], 'c2': [3, 1, 2, 0]}
q3_test_fit_X = pd.DataFrame(q3_test_fit_cols)
q3_test_fit_std = StdScalerByGroup().fit(q3_test_fit_X)

# test transform
q3_test_transform_cols = {'g': ['A', 'A', 'B', 'B'], 'c1': [1, 2, 3, 4], 'c2': [1, 2, 3, 4]}
q3_test_transform_X = pd.DataFrame(q3_test_transform_cols)
q3_test_transform_std = StdScalerByGroup().fit(q3_test_transform_X)
q3_test_transform_out = q3_test_transform_std.transform(q3_test_transform_X)

In [ ]:
# don't change this cell, but do run it 
q3_fit_data = pd.read_csv('data/toy.csv')

N = 2*10**6
a = np.random.choice(['A', 'B'], size=(N,1)).astype('object')
b = np.random.multivariate_normal([1, 2], [[1, 0],[0, 100]], size=N)
arr = np.hstack([a, b])
q3_transform_data = pd.DataFrame(arr)
q3_transform_data[1] = q3_transform_data[1].astype(float)
q3_transform_data[2] = q3_transform_data[2].astype(float)

### Question 4

`Pipeline`s are supposed to help you easily try different model configurations. Complete the implementation of the function `eval_toy_model`, which returns a hard-coded **list of tuples** consisting of the (RMSE, $R^2$) of three different modeling `Pipeline`s, fit and evaluated on the entire input dataset `data`. The three different `Pipeline`s are:
1. The `Pipeline` in Question 1.
1. The `Pipeline` in Question 2.
1. A `Pipeline` consisting of a linear regression model fit on features generated by applying `StdScalerByGroup` to `'c1'`, log-scaling `'c2'`, and applying `OneHotEncoder` to `'group'`.

## Congratulations! You've finished homework 6! 🎉🥳

As a reminder, all of the work you want to submit needs to be in `hw.py`.


---

To double-check your work, the cell below will rerun all of the autograder tests.